In [1]:
import string
import numpy as np
import tensorflow as tf
from itertools import chain
from keras.layers import Embedding, Dense, LSTM
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

2023-11-11 14:51:56.282621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
text = open('/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/current_corpora/en_wiki_extractor.txt', 'r', encoding='utf-8').read().splitlines()
text = [line.translate(str.maketrans("", "", string.punctuation)) for line in text]
text = [line.translate(str.maketrans("", "", string.digits)) for line in text]

In [3]:
vocab = list(set(chain(*(line.split() for line in text if line))))
vocab.insert(0, '<BOS>')
vocab.insert(1, '<EOS>')
vocab_size = len(vocab)

In [4]:
wd2idx = {word: i for i, word in enumerate(vocab)}
idx2wd = {i: word for i, word in enumerate(vocab)}
sequences = []

for i in range(2, len(vocab)):
    seq = [vocab[0], vocab[i], vocab[1]]
    sequences.append(seq)

In [19]:
sequences[:10]

[['<BOS>', 'rainfall', '<EOS>'],
 ['<BOS>', 'reorganisation', '<EOS>'],
 ['<BOS>', 'require', '<EOS>'],
 ['<BOS>', 'florins', '<EOS>'],
 ['<BOS>', 'sacaba', '<EOS>'],
 ['<BOS>', 'relay', '<EOS>'],
 ['<BOS>', 'nebulosity', '<EOS>'],
 ['<BOS>', 'interspersed', '<EOS>'],
 ['<BOS>', 'inuyasha', '<EOS>'],
 ['<BOS>', 'unconvincingly', '<EOS>']]

In [8]:
X = [[wd2idx[word] for word in sequence[:-1]] for sequence in sequences]
y = [[wd2idx[word] for word in sequence[1:]] for sequence in sequences]
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [13]:
tf.shape(X_train), tf.shape(y_train)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([33827,     2], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([33827,     2], dtype=int32)>)

In [ ]:
# sequences = np.asarray(sequences)[:, np.newaxis]
# X, y = sequences[:,:-1], sequences[:,-1]
# y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)
# seq_len = X.shape[1]

# max_len = len(max(text, key=lambda x: len(x.split())).split())

# predictors, label = input_seqs[:, :-1], input_seqs[:, -1]
# label = tf.keras.utils.to_categorical(label, num_classes=vocab_size)

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(text)
# vocab_size = len(tokenizer.word_index) + 1
# sequences = tokenizer.texts_to_sequences(text)

# sequences = tokenizer.texts_to_sequences(text)
# for line in text:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         seq = token_list[i - 1:i + 1]
#         input_seqs.append(seq)
        
# max_token_len = max([len(x) for x in sequences])
# sequences = pad_sequences(sequences, maxlen=max_token_len, padding='post')
# # input_seqs = np.array(input_seqs)
# X, y = sequences[:, :-1], sequences[:, -1]
# y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# split_index = int(0.8 * len(X))
# X_train, X_test = X[:split_index], X[split_index:]
# y_train, y_test = y[:split_index], y[split_index:]

In [6]:
batch_size = 32
num_epochs = 10
loss_function = SparseCategoricalCrossentropy()
optimizer = Adam()

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=None))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         5412608   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 42286)             5454894   
                                                                 
Total params: 10999086 (41.96 MB)
Trainable params: 10999086 (41.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.17_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/usr/local/Cellar/python@3.9/3.9.17_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/k/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/k/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/k/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 728, in start

  File "/Users/k/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/usr/local/Cellar/python@3.9/3.9.17_1/Frameworks/Python.fr

In [ ]:
# def calc_perplexity(model, test_data):
#     test_sequences = tokenizer.texts_to_sequences(test_data)
#     test_sequences = np.array(test_sequences)
#     test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_len - 1, padding='pre')
#     total_log_loss = 0
#     total_words = 0

#     for sequence in test_sequences:
#         input_seq = sequence[:-1]
#         target = sequence[-1]

#         predicted = model.predict(input_seq, verbose=0)[0]
#         if target != 0:
#             total_log_loss += np.log(predicted[int(target)])
#             total_words += 1

#     avg_log_loss = total_log_loss / total_words
#     perplexity = np.exp(-avg_log_loss)
#     return perplexity

# # Test data for calculating perplexity
# test_data = ["Finally, here is the third sentence.", "This is the first sentence."]
# perplexity = calc_perplexity(model, test_data)
# print(f"Perplexity: {perplexity}")
